In [0]:
import time
import pyspark

In [0]:
def reducer(x):
    key=x[0]
    values=x[1]
    min=key
    valuesList=[]
    listoutput=[]
    global newpaircounter
    for i in values:
        if i <min: 
            min=i
        valuesList.append(i)
    if min < key:
        listoutput.append((key,min))
        for j in valuesList:
            if min != j:
              listoutput.append((j,min))
              newpaircounter.add(1)
    return listoutput

In [0]:
path = "/FileStore/tables/RDD/" 
rdd_init = sc.textFile(path + "/facebook_combined.txt", use_unicode="False")
rdd=rdd_init.map(lambda x:(x.split(' ')))

In [0]:
path = "/FileStore/tables/RDD/" 
rdd_init = sc.textFile(path + "/large_twitch_edges.csv", use_unicode="False")
rdd=rdd_init.filter(lambda x :x != 'numeric_id_1,numeric_id_2').map(lambda x:(x.split(',')))

In [0]:
path = "/FileStore/tables/RDD/" 
rdd_init = sc.textFile(path + "/HR_edges.csv", use_unicode="False")
rdd=rdd_init.map(lambda x:(x.split(','))).filter(lambda x:x[0] != 'node_1')

In [0]:
path = "/FileStore/tables/RDD/" 
rdd_init = sc.textFile(path + "/artist_edges.csv", use_unicode="False")
rdd=rdd_init.filter(lambda x :x != 'node_1,node_2').map(lambda x:(x.split(',')))

In [0]:
path = "/FileStore/tables/RDD/" 
rdd_init = sc.textFile(path + "/web_Google.txt", use_unicode="False")
rdd=rdd_init.filter(lambda x:"#" not in x).map(lambda x:x.split("\t"))

In [0]:
path = "/FileStore/tables/RDD/" 
rdd_init = sc.textFile(path + "/soc_pokec_relationships.txt", use_unicode="False")
rdd=rdd_init.filter(lambda x:"#" not in x).map(lambda x:x.split("\t")).map(lambda x:(int(x[0]),int(x[1])))

In [0]:
#
boucle = True
t0=time.time()
iterationId=int(0)

while boucle == True:
  iterationId +=1
  newpaircounter=sc.accumulator(0)
  
  #map du ccf
  Map_rdd=rdd.flatMap(lambda x:(x,(x[1],x[0])))
  
  #fonction reduce du CCF Iterate RDD fichier de chiffres
  Reduce_rdd=Map_rdd.groupByKey().flatMap(lambda x:reducer(x)) #.filter(lambda x:x!=None).flatMap(lambda x:x)
  Reduce_rdd.count() 
  
  ##suite #fonction Reduce de CFF Dedup sur base RDD liste de chiffres pour vérifier la suppression des doublons de tuples
  rdd=Reduce_rdd.distinct().sortByKey()

  print("iteration n° :",iterationId, "nombre de paires créées : ", newpaircounter.value)
  if newpaircounter.value == 0:
    print("arret")
    boucle = False

t1=time.time()
print("c'est la fin; durée totale de :",round(t1-t0,3))
#rdd.collect()

#décompte du nombre de connected components

conclu=rdd.map(lambda x:(x[1],x[0])).groupByKey().map(lambda x:(x[0],list(x[1])))
nbre_cc=conclu.count()
inventaire=conclu.collect()

for j in range(0,nbre_cc):
    print("controle",j)
    print("connect component id :",conclu.collect()[j][0],"nombres de noeuds :",len(conclu.collect()[j][1])+1)
    

iteration n° : 1 nombre de paires créées :  118273064
iteration n° : 2 nombre de paires créées :  148592320
iteration n° : 3 nombre de paires créées :  124362218
iteration n° : 4 nombre de paires créées :  5689920
iteration n° : 5 nombre de paires créées :  408
iteration n° : 6 nombre de paires créées :  0
arret
c'est la fin; durée totale de : 6062.929
controle 0
connect component id : 1 nombres de noeuds : 1632803


In [0]:
for j in range(0,nbre_cc):
    print("controle",j)
    print("connect component id :",conclu.collect()[j][0],"nombres de noeuds :",len(conclu.collect()[j][1])+1)

In [0]:
def itRedCCF(pair):
  key, values = pair
  min = key
  valueL = []
  for value in values:
    if value < min:
       min = value
    valueL.append(value)
  if min < key:
    yield((key, min))
    for value in valueL:
      if min != value:
        accum.add(1)
        yield((value, min))

In [0]:
iterationID = 0
accum = sc.accumulator(0)
newPair = True

In [0]:
#contrôle avec un autre code

path = "/FileStore/tables/RDD/" 
rdd_init = sc.textFile(path + "/HR_edges.csv", use_unicode="False")
edges=rdd_init.map(lambda x:(x.split(','))).filter(lambda x:x[0] != 'node_1')
begin = time.time()

while newPair == True:
  iterationID += 1
  newPair = False
  accum.value = 0
  
  #CCF-iterate (MAP)
  firstJobMap = edges.flatMap(lambda e: (e, e[::-1]))
  
  #CCF-iterate (REDUCE)  
  firstJobReduce = firstJobMap.groupByKey().flatMap(lambda pair: itRedCCF(pair)).sortByKey()
  
  #CFF-debup (MAP & REDUCE)
  debupEdges = firstJobReduce.distinct()
  
  edges = debupEdges
  newPair = bool(accum.value)
  print("Iteration: ", iterationID, ", newPair? ", newPair)
    
#end of timer
ending = time.time() - begin
print("Execution time of data processing: ", ending)

results = list(map(lambda e: e[::-1], edges.collect()))
resultsAgg = sc.parallelize(results).groupByKey().map(lambda a: (a[0], list(a[1]))).collect()
print("Number of connected components in graph: ", len(resultsAgg))

In [0]:
resultsAgg[2][1]

In [0]:
edges.map(lambda x:(x[1],x[0])).groupByKey().mapValues(list).map(lambda x : (x[0], len(list(x[1]))+1)).collect()